# Song Recommender

In [1]:
import pandas as pd
import regex as re
from random import randint

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
import pickle
from time import sleep

from sklearn.cluster import KMeans

In [2]:
top_songs = pd.read_csv('top_songs.csv')
playlist_cluster_df = pd.read_csv('playlist_cluster_df.csv')

In [3]:
def open_spotify_connection():
    #Opens a new spotify connection
    secrets_file = open("secrets.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    return sp;

In [4]:
def remove_special_characters(text):
    # Remove special characters using a regular expression
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [5]:
def recommend_song(input_song, input_artist):

    # Convert the user input and song names to lowercase for case-insensitive comparison
    input_song_lower = input_song.lower()
    input_artist_lower = input_artist.lower()

    top_songs_lower = top_songs.apply(lambda x: x.str.lower().apply(remove_special_characters))
    top_songs_lower['song'] = top_songs_lower['song'].apply(remove_special_characters)

    # Check if the user-inputted song is in the top songs list (case-insensitive)
    if input_song_lower in top_songs_lower['song'].values:
        # Filter out the user-inputted song from the DataFrame
        top_songs_without_input = top_songs[
            top_songs_lower['song'] != input_song_lower
        ]

        # Recommend a random song from the filtered df
        recommended_song = top_songs_without_input.sample().squeeze()

        print(f"You might also like: {recommended_song['song']} by {recommended_song['artist']}")
        
    else:
        
        #Opening the spotify connection
        sp = open_spotify_connection()

        # Search for the user-inputted song and artist on Spotify
        search_results = sp.search(q=f"{input_song} {input_artist}", type='track', limit =1 )

        if not search_results['tracks']['items']:
            print("Song not found on Spotify. Please try another song.")
            return

        # Get the track ID for the user-inputted song
        if search_results['tracks']['total']!=0 :
            track_id = search_results['tracks']['items'][0]['id']
            #print(track_id)
        
        # Getting the audio features using the id
        new_data = sp.audio_features(track_id)
        #print(type(new_data[0]))
        data_df = json_normalize(new_data[0]) #normalizing the data to a df
        num_data_df = data_df.select_dtypes(include='number') #getting the numerical data only to fit
        #print(num_data_df)

        #Using pickle to get the existing scaler and kmean
        kmeans = pickle.load(open('kmeans.pkl', 'rb'))
        scaler = pickle.load(open('X_prep.pkl','rb'))
        X_prep = scaler.transform(num_data_df)
        predicted_cluster = kmeans.predict(X_prep)
        
        # Predict the cluster for the input song using the clustering model
        cluster = predicted_cluster[0]
        #print(cluster)
        
        # Filter songs in the same cluster from the playlist DataFrame and
        # Recommend a random song from the same cluster
        recommended_song = playlist_cluster_df[playlist_cluster_df['cluster']== cluster].sample().squeeze()
        # print(recommended_song)

        print(f"You might also like: {recommended_song['track.name']} by {recommended_song['name']}")

        # Introduce a respectful nap to avoid hitting rate limits
        sleep(randint(1, 3000) / 1000)


## Enter a song and its artist to get a recommendation!

In [6]:
# Get user input for song and artist
input_song = input("Enter a song: ")
input_artist = input("Enter the artist: ")

while True:
    recommend_song(input_song, input_artist)
    
    # Ask if the user wants another recommendation
    user_response = input("Do you want another recommendation? (yes/no): ").lower()
    
    if user_response not in ['yes', 'y']:
        # If the answer is not 'yes' or 'y', break out of the loop
        break

    # If the user wants another recommendation, get new input
    input_song = input("Enter a song: ")
    input_artist = input("Enter the artist: ")

print("Goodbye! Thanks for using the song recommender")

Enter a song: haul and drag
Enter the artist: el pony pisador
You might also like: Le monde est à pleurer by Jean Leloup
Do you want another recommendation? (yes/no): yes
Enter a song: happier than ever
Enter the artist: bilie eilish
You might also like: When a Man Loves a Woman by Michael Bolton
Do you want another recommendation? (yes/no): yes
Enter a song: respect
Enter the artist: aretha franklin
You might also like: Tiny Dancer by Elton John
Do you want another recommendation? (yes/no): no
Goodbye! Thanks for using the song recommender
